In [265]:
#setup data
import csv
import pandas as pd
import numpy as np

#define risk free rate(rfr) as the three month treasure yield (DGS3MO)
rfr = np.array((5244,1))
#define term spread as 10 year treasure interest(DGS10) minus 1 year treasure interest(DGS1)
ts = np.array((5244,1))
#define default spread as the yield gap between AAA(DAAA) and BBB(DBAA) grade bonds
ds = np.array((5244,1))
ones = np.ones((5244,1))[:,0]

data1 = pd.read_csv('spread.csv')
data2 = pd.read_csv('StockMarket.csv')
data = pd.merge(data1, data2, on='Date')
rfr = data['DGS3MO']
ds = data['DGS10'] - data['DGS1']
ts = data['DAAA'] - data['DBAA']
# print(rfr.shape)
# print(ds.shape)
# print(ts.shape)
# print(ones.shape)
A = np.stack((rfr, ds, ts, ones), axis = 1)
#delete the last data since there is no label for the next day
A = np.delete(A, 5243, 0)
# print(data)

#the yield of three major index: Dow Jones, SP 500 and Nasdaq
B_raw = np.stack((data['DJIA'], data['SP500'], data['NASDAQ']), axis = 1)
#Labeled as -1 if the market loss the next day, +1 otherwise 
B = np.zeros((5244, 3))
for i in range(5244):
    for j in range(3):
        B[i][j] = -1 if B_raw[i][j] < 0 else 1

#delete first label since there is no previous day data to predict
B= np.delete(B, 0, 0)      

In [266]:
#support vector machine using stochastic gradient descent, l1 regularization
def gs_svm1( A, d, la_array ):
    max_iter = 10**8
    tol = 10**(-6)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    m = A.shape[0]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    W = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            w_old = w
            s = random.randint(0, m - 1)#random index for stochastic  
            if (d[s]*np.dot(A[s], w)) < 1:
                w = w + tau * ( (np.atleast_2d(A[s]).T * d[s]) - each_lambda * np.sign(w) / m) 
            else:
                w = w - tau * each_lambda * np.sign(w) / m
            W[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return W

In [267]:
##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,525],[526,1050],[1051,1575],[1576,2100],[2101,2625],[2626,3150],[3151,3675],[3676,4200],[4201,4725],[4726,5243]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

lam_vals = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
er_sq = np.zeros((cases,1))
er_rate = np.zeros((cases,1))

In [269]:
# DJIA
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = gs_svm1(At,Bt[:,0].reshape((Bt.shape[0],1)),lam_vals)
    # calculate error
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            
    #get best lambda from above algorithm         
#     print(miniErIndex)
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,0], 2) / len(Bv2[:,0])
    error_vec2 = [0 if Bv2[j][0]==Bp2[j] else 1 for j in range(len(Bv2[:,0]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,0])

In [270]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06305884]
 [0.06047432]
 [0.05777429]
 [0.06224813]
 [0.0572697 ]
 [0.06011327]
 [0.05842759]
 [0.06142673]]
Average square error: 
 0.060099108085930636
Error rate for each case: 
 [[0.52190476]
 [0.48      ]
 [0.43809524]
 [0.50857143]
 [0.43047619]
 [0.47428571]
 [0.44208494]
 [0.4952381 ]]
Average error rate: 
 0.47383204633204634


In [271]:
# SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = gs_svm1(At,Bt[:,1].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][1]==Bp1[j] else 1 for j in range(len(Bv1[:,1]))]
        er[i] = sum(error_vec1) / len(Bv1[:,1])
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,1], 2) / len(Bv2[:,1])
    error_vec2 = [0 if Bv2[j][1]==Bp2[j] else 1 for j in range(len(Bv2[:,1]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,1])

In [272]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06419924]
 [0.05852302]
 [0.05764856]
 [0.06107131]
 [0.05752255]
 [0.06011327]
 [0.05817189]
 [0.06178009]]
Average square error: 
 0.05987874036523351
Error rate for each case: 
 [[0.54095238]
 [0.44952381]
 [0.43619048]
 [0.48952381]
 [0.43428571]
 [0.47428571]
 [0.43822394]
 [0.50095238]]
Average error rate: 
 0.470492277992278


In [279]:
# NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = gs_svm1(At,Bt[:,2].reshape((Bt.shape[0],1)),lam_vals)
    #calculate error
    er = np.zeros((26, 1))#error of the 26 lambda cases
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][2]==Bp1[j] else 1 for j in range(len(Bv1[:,2]))]
        er[i] = sum(error_vec1) / len(Bv1[:,2])
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,2], 2) / len(Bv2[:,2])
    error_vec2 = [0 if Bv2[j][2]==Bp2[j] else 1 for j in range(len(Bv2[:,2]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,2])

[ 0.15118229  0.1741951  -0.11715783  0.11772759]
[ 0.13689789  0.12834641 -0.08757795  0.09276897]
[ 0.1075475   0.18260347 -0.11269674  0.11467455]
[ 0.09506978  0.20215531 -0.10616361  0.11292075]
[ 0.13902526  0.08744171 -0.07169355  0.08261886]
[ 0.14175003  0.08621996 -0.07020868  0.07307276]
[ 0.13581868  0.18045474 -0.09304268  0.09747154]
[ 0.12924061  0.18256701 -0.10478768  0.11092147]


In [281]:
print("The classifier based on Nasdaq index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Nasdaq index:
Square error for each case: 
 [[0.06305884]
 [0.06023386]
 [0.05739626]
 [0.06178009]
 [0.0563758 ]
 [0.05839889]
 [0.05829988]
 [0.05987137]]
Average square error: 
 0.05942687522414974
Error rate for each case: 
 [[0.52190476]
 [0.47619048]
 [0.43238095]
 [0.50095238]
 [0.41714286]
 [0.44761905]
 [0.44015444]
 [0.47047619]]
Average error rate: 
 0.4633526383526384


In [282]:
#support vector machine using stochastic gradient descent, l2 regularization
def gs_svm2( A, d, la_array ):
    max_iter = 10**8
    tol = 10**(-3) / 5000
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    m = A.shape[0]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    W = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            w_old = w
            s = random.randint(0, m - 1)#random index for stochastic  
            if (d[s]*np.dot(A[s], w)) < 1:
                w = w + tau * ( (np.atleast_2d(A[s]).T * d[s]) - each_lambda * w/ m) 
            else:
                w = w - tau * each_lambda * w / m
            W[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return W

In [295]:
#Dow Jones
for j in range(cases):
        # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,0]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,0]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,0]
    
    #train classifiers
    W = gs_svm2(At,Bt[:,0].reshape((Bt.shape[0],1)),lam_vals)
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
#         Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = W[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)


[ 0.11867052  0.15116458 -0.17568614  0.12859155]
[ 0.0777986   0.1670625  -0.15568683  0.11496373]
[ 0.04340869  0.18051815 -0.14543832  0.11086273]
[ 0.08302872  0.20581011 -0.12675097  0.13349934]
[ 0.05204252  0.22350223 -0.1227453   0.12612732]
[ 0.13755499  0.1752365  -0.11998646  0.10605705]
[ 0.09361674  0.2087016  -0.12123533  0.10080242]
[ 0.15211795  0.1735704  -0.09791106  0.11143615]


In [296]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06305884]
 [0.06047432]
 [0.05777429]
 [0.06224813]
 [0.0572697 ]
 [0.06011327]
 [0.05842759]
 [0.06142673]]
Average square error: 
 0.060099108085930636
Error rate for each case: 
 [[0.52190476]
 [0.48      ]
 [0.43809524]
 [0.50857143]
 [0.43047619]
 [0.47428571]
 [0.44208494]
 [0.4952381 ]]
Average error rate: 
 0.47383204633204634


In [286]:
#SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,1]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,1]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,1]
    
    #train classifiers
    W = gs_svm2(At,Bt[:,1].reshape((Bt.shape[0],1)),lam_vals)
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = W[:,i]
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

In [287]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06419924]
 [0.05852302]
 [0.05764856]
 [0.06107131]
 [0.05752255]
 [0.06011327]
 [0.05817189]
 [0.06178009]]
Average square error: 
 0.05987874036523351
Error rate for each case: 
 [[0.54095238]
 [0.44952381]
 [0.43619048]
 [0.48952381]
 [0.43428571]
 [0.47428571]
 [0.43822394]
 [0.50095238]]
Average error rate: 
 0.470492277992278


In [297]:
#NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,2]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,2]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,2]
    
    #train classifiers
    W = gs_svm2(At,Bt[:,2].reshape((Bt.shape[0],1)),lam_vals)
    
    for i in range(26):
        bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = W[:,i]
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

In [298]:
print("The classifier based on NASDAQ index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on NASDAQ index:
Square error for each case: 
 [[0.06305884]
 [0.06023386]
 [0.05739626]
 [0.06178009]
 [0.0563758 ]
 [0.05839889]
 [0.05829988]
 [0.05987137]]
Average square error: 
 0.05942687522414974
Error rate for each case: 
 [[0.52190476]
 [0.47619048]
 [0.43238095]
 [0.50095238]
 [0.41714286]
 [0.44761905]
 [0.44015444]
 [0.47047619]]
Average error rate: 
 0.4633526383526384
